In [1]:
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler, random_split
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import xml.etree.ElementTree as ET
from torch.autograd import Variable
import time
from tqdm import tqdm

In [2]:
img_count = 0
for folder in os.listdir('data'):
    for _ in os.listdir('data/' + folder):
        img_count += 1
print('No. of Images: {}'.format(img_count))

No. of Images: 20580


In [3]:
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=315, scale=(0.95, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=299),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    'test':
    transforms.Compose([
        transforms.Resize(size=299),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [4]:
batch_size = 128

all_data = datasets.ImageFolder(root='data')
train_data_len = int(len(all_data)*0.8)
valid_data_len = int((len(all_data) - train_data_len)/2)
test_data_len = int(len(all_data) - train_data_len - valid_data_len)
train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
train_data.dataset.transform = image_transforms['train']
val_data.dataset.transform = image_transforms['test']
test_data.dataset.transform = image_transforms['test']
print(len(train_data), len(val_data), len(test_data))

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

16464 2058 2058


In [5]:
trainiter = iter(train_loader)
features, labels = next(trainiter)
print(features.shape, labels.shape)

torch.Size([128, 3, 299, 299]) torch.Size([128])


In [6]:
model = models.inception_v3(pretrained=True)
model.aux_logits=False
# Freeze early layers
for param in model.parameters():
    param.requires_grad = False
n_classes = 120
n_inputs = model.fc.in_features
# n_inputs will be 4096 for this case
# Add on classifier
model.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, n_classes),
    nn.LogSoftmax(dim=1))

In [7]:
model.class_to_idx = all_data.class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()}

list(model.idx_to_class.items())

[(0, 'n02085620-Chihuahua'),
 (1, 'n02085782-Japanese_spaniel'),
 (2, 'n02085936-Maltese_dog'),
 (3, 'n02086079-Pekinese'),
 (4, 'n02086240-Shih-Tzu'),
 (5, 'n02086646-Blenheim_spaniel'),
 (6, 'n02086910-papillon'),
 (7, 'n02087046-toy_terrier'),
 (8, 'n02087394-Rhodesian_ridgeback'),
 (9, 'n02088094-Afghan_hound'),
 (10, 'n02088238-basset'),
 (11, 'n02088364-beagle'),
 (12, 'n02088466-bloodhound'),
 (13, 'n02088632-bluetick'),
 (14, 'n02089078-black-and-tan_coonhound'),
 (15, 'n02089867-Walker_hound'),
 (16, 'n02089973-English_foxhound'),
 (17, 'n02090379-redbone'),
 (18, 'n02090622-borzoi'),
 (19, 'n02090721-Irish_wolfhound'),
 (20, 'n02091032-Italian_greyhound'),
 (21, 'n02091134-whippet'),
 (22, 'n02091244-Ibizan_hound'),
 (23, 'n02091467-Norwegian_elkhound'),
 (24, 'n02091635-otterhound'),
 (25, 'n02091831-Saluki'),
 (26, 'n02092002-Scottish_deerhound'),
 (27, 'n02092339-Weimaraner'),
 (28, 'n02093256-Staffordshire_bullterrier'),
 (29, 'n02093428-American_Staffordshire_terrier'),


In [8]:
model.cuda()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [49]:
def train(model,
         criterion,
         optimizer,
         train_loader,
         val_loader,
         save_location,
         early_stop=3,
         n_epochs=20,
         print_every=2):
    
    since = time.time()
    #Initializing some variables
    valid_loss_min = np.Inf
    stop_count = 0
    valid_max_acc = 0
    history = []
    model.epochs = 0
    

    #Loop starts here
    for epoch in range(n_epochs):
        
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)
        total_loss_train = 0
        total_loss_val = 0
        train_loss = 0
        valid_loss = 0

        train_acc = 0
        valid_acc = 0
        cntr_val = 0
        cntr_tr = 0
        corrects_train = 0
        corrects_val = 0
        
        

        model.train()
        
        
        t = tqdm(train_loader, mininterval=1, desc='-(Training)', leave=False)
        
        for batch in t:
            data, label = batch
            
            if torch.cuda.is_available():
                data = Variable(data.cuda())
                label = Variable(label.cuda())

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            cntr_tr += 1

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)
            
            description = "Loss: " + str(loss.item())
            t.set_description(description)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1) # first output gives the max value in the row(not what we want), second output gives index of the highest val
            correct_tensor = pred.eq(label.data.view_as(pred)) # using the index of the predicted outcome above, torch.eq() will check prediction index against label index to see if prediction is correct(returns 1 if correct, 0 if not)
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor)) #tensor must be float to calc average
            corrects_train += torch.sum(pred == label.data)
            train_acc += accuracy.item() * data.size(0)
            
            total_loss_train += loss.item()

        avg_loss = total_loss_train / float(cntr_tr)
        print('The average loss in TRAINING is', avg_loss)
        
        print('Accuracy TRAINING: {}/{} ({:.0f}%)\n'.format(corrects_train, len(train_loader.dataset), 100. * corrects_train / len(train_loader.dataset)))

        model.epochs += 1
        with torch.no_grad():
            model.eval()
            t = tqdm(val_loader, mininterval=1, desc='-(Validation)', leave=False)
    
            for batch in t:
                data, label = batch
                if torch.cuda.is_available():
                    data = Variable(data.cuda())
                    label = Variable(label.cuda())
      
                output = model(data)
                loss = criterion(output, label)
                valid_loss += loss.item() * data.size(0)
                cntr_val += 1
                
                description = "Loss: " + str(loss.item())
                t.set_description(description)

                _, pred = torch.max(output, dim=1)
                correct_tensor = pred.eq(label.data.view_as(pred))
                corrects_val += torch.sum(pred == label.data)
                accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
                valid_acc += accuracy.item() * data.size(0)
                
                total_loss_val += loss.item()

            avg_loss = total_loss_val / float(cntr_val)
            print('The average loss in VALIDATIONis', avg_loss)
            print('Accuracy VALIDATION: {}/{} ({:.0f}%)\n'.format(corrects_val, len(val_loader.dataset), 100. * corrects_val / len(val_loader.dataset)))

            train_loss = train_loss / len(train_loader.dataset)
            valid_loss = valid_loss / len(val_loader.dataset)

            train_acc = train_acc / len(train_loader.dataset)
            valid_acc = valid_acc / len(val_loader.dataset)

            history.append([train_loss, valid_loss, train_acc, valid_acc])

            if (epoch + 1) % print_every == 0:
                print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}')
                print(f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%')

            if valid_loss < valid_loss_min:
                torch.save({
                    'state_dict': model.state_dict(),
                    'idx_to_class': model.idx_to_class
                }, save_location)
                stop_count = 0
                valid_loss_min = valid_loss
                valid_best_acc = valid_acc
                best_epoch = epoch

            else:
                stop_count += 1

                # Below is the case where we handle the early stop case
                if stop_count >= early_stop:
                    print(f'\nEarly Stopping Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
                    model.load_state_dict(torch.load(save_location)['state_dict'])
                    model.optimizer = optimizer
                    history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_acc','valid_acc'])
                    return model, history

    model.optimizer = optimizer
    print(f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')

    history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    chkpt_name = "model.chkpt"
    torch.save(model.state_dict(), chkpt_name)
    
    return model, history

In [48]:
train(model.cuda(),
         criterion,
         optimizer,
         train_loader,
         val_loader,
         save_location='./dog_inception.pt',
         early_stop=3,
         n_epochs=5,
         print_every=1)









-(Training):   0%|          | 0/129 [00:00<?, ?it/s]

Epoch 0/4
----------










Loss: 0.7725688815116882:   0%|          | 0/129 [00:00<?, ?it/s]







Loss: 0.8576922416687012:   0%|          | 0/129 [00:01<?, ?it/s]







Loss: 0.8576922416687012:   2%|▏         | 2/129 [00:01<01:55,  1.10it/s]







Loss: 0.5273559093475342:   2%|▏         | 2/129 [00:02<01:55,  1.10it/s]







Loss: 0.7687163352966309:   2%|▏         | 2/129 [00:03<01:55,  1.10it/s]







Loss: 0.7687163352966309:   3%|▎         | 4/129 [00:03<01:53,  1.10it/s]







Loss: 0.8670000433921814:   3%|▎         | 4/129 [00:04<01:53,  1.10it/s]







Loss: 0.5316222906112671:   3%|▎         | 4/129 [00:05<01:53,  1.10it/s]







Loss: 0.5316222906112671:   5%|▍         | 6/129 [00:05<01:50,  1.11it/s]







Loss: 0.7792235016822815:   5%|▍         | 6/129 [00:06<01:50,  1.11it/s]







Loss: 0.6572646498680115:   5%|▍         | 6/129 [00:07<01:50,  1.11it/s]







Loss: 0.6572646498680115:   6%|▌         | 8/129 [00:07<01:48,  1.12it/s]







Loss: 0.4606449604034424:   6%|▌    

Loss: 0.6441424489021301:  79%|███████▉  | 102/129 [01:33<00:23,  1.13it/s]







Loss: 0.6441424489021301:  81%|████████  | 104/129 [01:33<00:22,  1.13it/s]







Loss: 0.7220457196235657:  81%|████████  | 104/129 [01:34<00:22,  1.13it/s]







Loss: 0.8218316435813904:  81%|████████  | 104/129 [01:35<00:22,  1.13it/s]







Loss: 0.8218316435813904:  82%|████████▏ | 106/129 [01:35<00:20,  1.13it/s]







Loss: 0.844851016998291:  82%|████████▏ | 106/129 [01:36<00:20,  1.13it/s] 







Loss: 0.627334475517273:  82%|████████▏ | 106/129 [01:36<00:20,  1.13it/s]







Loss: 0.627334475517273:  84%|████████▎ | 108/129 [01:36<00:18,  1.12it/s]







Loss: 0.6354551911354065:  84%|████████▎ | 108/129 [01:37<00:18,  1.12it/s]







Loss: 0.7121210098266602:  84%|████████▎ | 108/129 [01:38<00:18,  1.12it/s]







Loss: 0.7121210098266602:  85%|████████▌ | 110/129 [01:38<00:16,  1.12it/s]







Loss: 0.7790817022323608:  85%|████████▌ | 110/129 [01:39<00:16,  1.12it/s]







Loss: 

The average loss in TRAINING is 0.6854978141396545
Accuracy TRAINING: 12996/16464 (78%)



...

Loss: 0.6024484634399414:  79%|███████▉  | 102/129 [01:32<00:23,  1.14it/s]







Loss: 0.6024484634399414:  81%|████████  | 104/129 [01:32<00:22,  1.13it/s]







Loss: 0.48422256112098694:  81%|████████  | 104/129 [01:33<00:22,  1.13it/s]







Loss: 0.6351813673973083:  81%|████████  | 104/129 [01:34<00:22,  1.13it/s] 







Loss: 0.6351813673973083:  82%|████████▏ | 106/129 [01:34<00:20,  1.13it/s]







Loss: 0.5548518896102905:  82%|████████▏ | 106/129 [01:35<00:20,  1.13it/s]







Loss: 0.5945453643798828:  82%|████████▏ | 106/129 [01:36<00:20,  1.13it/s]







Loss: 0.5945453643798828:  84%|████████▎ | 108/129 [01:36<00:18,  1.13it/s]







Loss: 0.5922344923019409:  84%|████████▎ | 108/129 [01:36<00:18,  1.13it/s]







Loss: 0.625360369682312:  84%|████████▎ | 108/129 [01:37<00:18,  1.13it/s] 







Loss: 0.625360369682312:  85%|████████▌ | 110/129 [01:37<00:16,  1.13it/s]







Loss: 0.7615519165992737:  85%|████████▌ | 110/129 [01:38<00:16,  1.13it/s]







Los

The average loss in TRAINING is 0.5947957334592361
Accuracy TRAINING: 13435/16464 (81%)











Loss: 0.6252379417419434:   0%|          | 0/17 [00:00<?, ?it/s]







Loss: 0.7021876573562622:   0%|          | 0/17 [00:01<?, ?it/s]







Loss: 0.7021876573562622:  12%|█▏        | 2/17 [00:01<00:13,  1.11it/s]







Loss: 0.5257655382156372:  12%|█▏        | 2/17 [00:02<00:13,  1.11it/s]







Loss: 0.6405477523803711:  12%|█▏        | 2/17 [00:03<00:13,  1.11it/s]







Loss: 0.6405477523803711:  24%|██▎       | 4/17 [00:03<00:11,  1.12it/s]







Loss: 0.7207270860671997:  24%|██▎       | 4/17 [00:04<00:11,  1.12it/s]







Loss: 0.832454264163971:  24%|██▎       | 4/17 [00:05<00:11,  1.12it/s] 







Loss: 0.832454264163971:  35%|███▌      | 6/17 [00:05<00:09,  1.13it/s]







Loss: 1.1878796815872192:  35%|███▌      | 6/17 [00:06<00:09,  1.13it/s]







Loss: 0.7160012125968933:  35%|███▌      | 6/17 [00:06<00:09,  1.13it/s]







Loss: 0.7160012125968933:  47%|████▋     | 8/17 [00:06<00:07,  1.14it/s]







Loss: 0.7893608808517456:  47%|████▋     | 8/17 [

The average loss in VALIDATIONis 0.746054561699138
Accuracy VALIDATION: 1613/2058 (78%)


Epoch: 4 	Training Loss: 0.5949 	Validation Loss: 0.7368
		Training Accuracy: 81.60%	 Validation Accuracy: 78.38%

Best epoch: 4 with loss: 0.74 and acc: 78.38%


(Inception3(
   (Conv2d_1a_3x3): BasicConv2d(
     (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
     (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   )
   (Conv2d_2a_3x3): BasicConv2d(
     (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
     (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   )
   (Conv2d_2b_3x3): BasicConv2d(
     (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   )
   (Conv2d_3b_1x1): BasicConv2d(
     (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   )
   (Conv2d_4a_3x3): BasicConv2d(
     (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
     (bn): BatchNorm2d(192, eps=0.001, momentum

In [61]:
def evaluate(valid_data_iterator, criterion, model):
    model.eval()
    t = tqdm(valid_data_iterator, mininterval=1, desc='-(Evaluate)', leave=False)
    total_loss = 0
    cntr = 0
    corrects =0 
    
    for batch in t:
        inputs, labels = batch
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        description = "Loss: " + str(loss.item())
        t.set_description(description)
        cntr += 1
        corrects += torch.sum(preds == labels.data)
        total_loss += loss.item()

    avg_loss = total_loss / float(cntr)
    
    print('The average loss is', avg_loss)

    print('Accuracy: {}/{} ({:.0f}%)\n'.format(corrects, len(val_loader.dataset), 100. * corrects / len(val_loader.dataset)))

In [51]:
chkpt_name = "model.chkpt"
model.load_state_dict(torch.load(chkpt_name))
evaluate(test_loader, criterion, model.cuda())









-(Evaluate):   0%|          | 0/17 [00:00<?, ?it/s]







Loss: 0.3312954604625702:   0%|          | 0/17 [00:00<?, ?it/s]







Loss: 0.4074103534221649:   0%|          | 0/17 [00:01<?, ?it/s]







Loss: 0.4074103534221649:  12%|█▏        | 2/17 [00:01<00:12,  1.16it/s]







Loss: 0.36873647570610046:  12%|█▏        | 2/17 [00:02<00:12,  1.16it/s]







Loss: 0.3094480037689209:  12%|█▏        | 2/17 [00:03<00:12,  1.16it/s] 







Loss: 0.3094480037689209:  24%|██▎       | 4/17 [00:03<00:11,  1.17it/s]







Loss: 0.4213830232620239:  24%|██▎       | 4/17 [00:04<00:11,  1.17it/s]







Loss: 0.39208903908729553:  24%|██▎       | 4/17 [00:05<00:11,  1.17it/s]







Loss: 0.39208903908729553:  35%|███▌      | 6/17 [00:05<00:09,  1.16it/s]







Loss: 0.3469747006893158:  35%|███▌      | 6/17 [00:06<00:09,  1.16it/s] 







Loss: 0.44204282760620117:  35%|███▌      | 6/17 [00:06<00:09,  1.16it/s]







Loss: 0.44204282760620117:  47%|████▋     | 8/17 [00:06<00:07, 

The average loss is 0.3640129846685073
Accuracy: 1845/2058 (89%)



0.3640129846685073

In [52]:
def results(model, test_loader, criterion):
  
    classes = []
    acc_results = np.zeros(len(test_loader.dataset))
    i = 0

    model.eval()
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.cuda(), labels.cuda()
            output = model(data)

            for pred, true in zip(output, labels):
                _, pred = pred.unsqueeze(0).topk(1)
                correct = pred.eq(true.unsqueeze(0))
                acc_results[i] = correct.cpu()
                classes.append(model.idx_to_class[true.item()][10:])
                i+=1

    results = pd.DataFrame({
      'class': classes,
      'results': acc_results    
    })
    results = results.groupby(classes).mean()

    return results

In [53]:
print(results(model, test_loader, criterion))

                                 results
Afghan_hound                    1.000000
African_hunting_dog             1.000000
Airedale                        0.785714
American_Staffordshire_terrier  0.826087
Appenzeller                     0.684211
Australian_terrier              0.782609
Bedlington_terrier              0.944444
Bernese_mountain_dog            1.000000
Blenheim_spaniel                1.000000
Border_collie                   0.636364
Border_terrier                  0.909091
Boston_bull                     0.909091
Bouvier_des_Flandres            0.812500
Brabancon_griffon               1.000000
Brittany_spaniel                1.000000
Cardigan                        0.736842
Chesapeake_Bay_retriever        0.928571
Chihuahua                       0.750000
Dandie_Dinmont                  0.833333
Doberman                        1.000000
English_foxhound                0.789474
English_setter                  0.888889
English_springer                1.000000
EntleBucher     

In [65]:
model.load_state_dict(torch.load(chkpt_name))
evaluate(test_loader, criterion, model.cuda())









-(Evaluate):   0%|          | 0/17 [00:00<?, ?it/s]







Loss: 0.4038044214248657:   0%|          | 0/17 [00:00<?, ?it/s]







Loss: 0.41939711570739746:   0%|          | 0/17 [00:01<?, ?it/s]







Loss: 0.41939711570739746:  12%|█▏        | 2/17 [00:01<00:12,  1.19it/s]







Loss: 0.43787434697151184:  12%|█▏        | 2/17 [00:02<00:12,  1.19it/s]







Loss: 0.29076486825942993:  12%|█▏        | 2/17 [00:03<00:12,  1.19it/s]







Loss: 0.29076486825942993:  24%|██▎       | 4/17 [00:03<00:11,  1.17it/s]







Loss: 0.40296241641044617:  24%|██▎       | 4/17 [00:04<00:11,  1.17it/s]







Loss: 0.2579902708530426:  24%|██▎       | 4/17 [00:05<00:11,  1.17it/s] 







Loss: 0.2579902708530426:  35%|███▌      | 6/17 [00:05<00:09,  1.16it/s]







Loss: 0.3306899666786194:  35%|███▌      | 6/17 [00:06<00:09,  1.16it/s]







Loss: 0.38266339898109436:  35%|███▌      | 6/17 [00:06<00:09,  1.16it/s]







Loss: 0.38266339898109436:  47%|████▋     | 8/17 [00:06<00:07

The average loss is 0.36057984127717857
Accuracy: 1845/2058 (89%)



In [63]:
best_model = "dog_inception.pt"
checkpoint = torch.load(best_model)
model.load_state_dict(checkpoint['state_dict'])
evaluate(test_loader, criterion, model.cuda())









-(Evaluate):   0%|          | 0/17 [00:00<?, ?it/s]







Loss: 0.8367533683776855:   0%|          | 0/17 [00:00<?, ?it/s]







Loss: 0.6421194076538086:   0%|          | 0/17 [00:01<?, ?it/s]







Loss: 0.6421194076538086:  12%|█▏        | 2/17 [00:01<00:12,  1.16it/s]







Loss: 0.7463072538375854:  12%|█▏        | 2/17 [00:02<00:12,  1.16it/s]







Loss: 0.6309620141983032:  12%|█▏        | 2/17 [00:03<00:12,  1.16it/s]







Loss: 0.6309620141983032:  24%|██▎       | 4/17 [00:03<00:11,  1.16it/s]







Loss: 0.6278551816940308:  24%|██▎       | 4/17 [00:04<00:11,  1.16it/s]







Loss: 0.8997787833213806:  24%|██▎       | 4/17 [00:05<00:11,  1.16it/s]







Loss: 0.8997787833213806:  35%|███▌      | 6/17 [00:05<00:09,  1.17it/s]







Loss: 0.6606113910675049:  35%|███▌      | 6/17 [00:06<00:09,  1.17it/s]







Loss: 0.6739521026611328:  35%|███▌      | 6/17 [00:06<00:09,  1.17it/s]







Loss: 0.6739521026611328:  47%|████▋     | 8/17 [00:06<00:07,  1.14it

The average loss is 0.7695973620695227
Accuracy: 1620/2058 (78%)

